# Data Preparation Pipeline - Preparación Final para ML
**Sistema de Anotación de Video - Entrega 2**

Este notebook implementa la preparación completa de datos para entrenamiento de modelos ML.

## Proceso Completo:
1. **Cargar dataset aumentado** (11,406 frames balanceados)
2. **Feature Engineering** (crear características avanzadas)
3. **Pipeline de preprocessing** (normalización, encoding)
4. **Data splits** estratificados (train/val/test)
5. **Validación y documentación** final

## Input:
- `data/augmented/landmarks_final_augmented.csv` (del Notebook 3)

## Output:
- Datasets listos para Tomas (train/val/test)
- Pipeline de preprocessing automatizado

---


## Paso 1: Instalación y Setup
Instalar librerías necesarias para feature engineering y preprocessing.


In [2]:
# Instalar dependencias para data preparation
!pip install pandas numpy matplotlib seaborn scikit-learn joblib

# Imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler, LabelEncoder, MinMaxScaler
from sklearn.model_selection import train_test_split, StratifiedShuffleSplit
from sklearn.decomposition import PCA
from sklearn.feature_selection import SelectKBest, f_classif
import joblib
import json
import os
import sklearn
from pathlib import Path
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

print("✅ Librerías para data preparation cargadas")
print(f"📊 Pandas: {pd.__version__}")
print(f"🔬 Scikit-learn: {sklearn.__version__}")
print(f"💾 Joblib: {joblib.__version__}")


✅ Librerías para data preparation cargadas
📊 Pandas: 2.2.2
🔬 Scikit-learn: 1.6.1
💾 Joblib: 1.5.2


## Paso 2: Cargar Dataset Aumentado
Cargar el dataset balanceado y aumentado desde el Notebook 3.


In [13]:
# CARGAR DATASET AUMENTADO
def load_augmented_dataset():
    """Cargar dataset aumentado del Notebook 3"""
    print("📂 CARGANDO DATASET AUMENTADO")
    print("=" * 40)

    # Use the correct file path
    dataset_path = "/content/landmarks_final_augmented.csv"

    if not os.path.exists(dataset_path):
        print(f"❌ Dataset no encontrado: {dataset_path}")
        print("💡 Asegúrate de ejecutar Notebook 3 primero")
        return None

    # Cargar datos
    df = pd.read_csv(dataset_path)

    print(f"✅ DATASET CARGADO:")
    print(f"   📊 Frames: {len(df):,}")
    print(f"   📋 Columnas: {len(df.columns)}")
    print(f"   🎬 Videos únicos: {df['video_file'].nunique():,}")
    print(f"   🎯 Actividades: {df['activity'].nunique()}")

    # Verificar distribución
    print(f"\n📊 DISTRIBUCIÓN POR ACTIVIDAD:")
    activity_counts = df['activity'].value_counts()
    for activity, count in activity_counts.items():
        pct = count / len(df) * 100
        print(f"   {activity.replace('_', ' ').title()}: {count:,} frames ({pct:.1f}%)")

    balance_ratio = activity_counts.min() / activity_counts.max()
    print(f"\n⚖️ Balance verificado: {balance_ratio:.3f}")

    # Verificar tipos de augmentation
    if 'source' in df.columns:
        print(f"\n🔧 TÉCNICAS DE AUGMENTATION:")
        source_counts = df['source'].value_counts()
        for source, count in source_counts.items():
            pct = count / len(df) * 100
            print(f"   {source.replace('_', ' ').title()}: {count:,} frames ({pct:.1f}%)")

    return df

# Cargar dataset aumentado
df_augmented = load_augmented_dataset()

if df_augmented is not None:
    print(f"\n✅ DATASET AUMENTADO CARGADO CORRECTAMENTE")
    print(f"🎯 Listo para Feature Engineering")
else:
    print(f"\n❌ Error cargando dataset aumentado")

📂 CARGANDO DATASET AUMENTADO
✅ DATASET CARGADO:
   📊 Frames: 11,406
   📋 Columnas: 69
   🎬 Videos únicos: 5,997
   🎯 Actividades: 5

📊 DISTRIBUCIÓN POR ACTIVIDAD:
   Caminar Hacia: 2,464 frames (21.6%)
   Caminar Regreso: 2,368 frames (20.8%)
   Girar: 2,278 frames (20.0%)
   Ponerse Pie: 2,227 frames (19.5%)
   Sentarse: 2,069 frames (18.1%)

⚖️ Balance verificado: 0.840

🔧 TÉCNICAS DE AUGMENTATION:
   Smote Synthetic: 5,445 frames (47.7%)
   Real Video: 4,575 frames (40.1%)
   Temporal Augmentation: 886 frames (7.8%)
   Spatial Augmentation: 500 frames (4.4%)

✅ DATASET AUMENTADO CARGADO CORRECTAMENTE
🎯 Listo para Feature Engineering


## Paso 3: Feature Engineering - Características Geométricas
Crear características geométricas avanzadas a partir de landmarks existentes.


In [14]:
# FEATURE ENGINEERING - CARACTERÍSTICAS GEOMÉTRICAS
class GeometricFeatureEngineer:
    """Crear características geométricas desde landmarks"""

    def __init__(self):
        # Definir landmarks principales
        self.landmarks = {
            'L_shoulder': ['L_shoulder_x', 'L_shoulder_y', 'L_shoulder_z'],
            'R_shoulder': ['R_shoulder_x', 'R_shoulder_y', 'R_shoulder_z'],
            'L_elbow': ['L_elbow_x', 'L_elbow_y', 'L_elbow_z'],
            'R_elbow': ['R_elbow_x', 'R_elbow_y', 'R_elbow_z'],
            'L_wrist': ['L_wrist_x', 'L_wrist_y', 'L_wrist_z'],
            'R_wrist': ['R_wrist_x', 'R_wrist_y', 'R_wrist_z'],
            'L_hip': ['L_hip_x', 'L_hip_y', 'L_hip_z'],
            'R_hip': ['R_hip_x', 'R_hip_y', 'R_hip_z'],
            'L_knee': ['L_knee_x', 'L_knee_y', 'L_knee_z'],
            'R_knee': ['R_knee_x', 'R_knee_y', 'R_knee_z'],
            'L_ankle': ['L_ankle_x', 'L_ankle_y', 'L_ankle_z'],
            'R_ankle': ['R_ankle_x', 'R_ankle_y', 'R_ankle_z']
        }

    def create_geometric_features(self, df):
        """Crear todas las características geométricas"""
        print("🔶 CREANDO CARACTERÍSTICAS GEOMÉTRICAS")
        print("=" * 45)

        df_geo = df.copy()
        features_created = []

        # 1. Distancias entre landmarks
        distances = self._create_distance_features(df_geo)
        features_created.extend(distances)

        # 2. Ángulos articulares
        angles = self._create_angle_features(df_geo)
        features_created.extend(angles)

        # 3. Ratios corporales
        ratios = self._create_ratio_features(df_geo)
        features_created.extend(ratios)

        # 4. Centro de masa
        center_features = self._create_center_mass_features(df_geo)
        features_created.extend(center_features)

        print(f"\n✅ CARACTERÍSTICAS GEOMÉTRICAS CREADAS:")
        print(f"   📊 Total nuevas features: {len(features_created)}")
        print(f"   📐 Dimensiones: {df_geo.shape[0]:,} × {df_geo.shape[1]}")

        return df_geo, features_created

    def _create_distance_features(self, df):
        """Crear características de distancias"""
        print("   📏 Creando distancias entre landmarks...")

        distances_created = []

        # Distancias corporales importantes
        distance_pairs = [
            ('L_shoulder', 'R_shoulder', 'shoulder_width'),
            ('L_hip', 'R_hip', 'hip_width'),
            ('L_shoulder', 'L_hip', 'L_torso_length'),
            ('R_shoulder', 'R_hip', 'R_torso_length'),
            ('L_hip', 'L_knee', 'L_thigh_length'),
            ('R_hip', 'R_knee', 'R_thigh_length'),
            ('L_knee', 'L_ankle', 'L_shin_length'),
            ('R_knee', 'R_ankle', 'R_shin_length')
        ]

        for lm1, lm2, feature_name in distance_pairs:
            if all(col in df.columns for col in self.landmarks[lm1]) and \
               all(col in df.columns for col in self.landmarks[lm2]):

                # Calcular distancia euclidiana 3D
                x1, y1, z1 = df[self.landmarks[lm1]].T.values
                x2, y2, z2 = df[self.landmarks[lm2]].T.values

                distance = np.sqrt((x2-x1)**2 + (y2-y1)**2 + (z2-z1)**2)
                df[feature_name] = distance
                distances_created.append(feature_name)

        print(f"      ✅ {len(distances_created)} distancias creadas")
        return distances_created

    def _create_angle_features(self, df):
        """Crear características de ángulos articulares"""
        print("   📐 Creando ángulos articulares...")

        angles_created = []

        # Ángulos articulares importantes
        angle_definitions = [
            ('L_shoulder', 'L_elbow', 'L_wrist', 'L_elbow_angle'),
            ('R_shoulder', 'R_elbow', 'R_wrist', 'R_elbow_angle'),
            ('L_hip', 'L_knee', 'L_ankle', 'L_knee_angle'),
            ('R_hip', 'R_knee', 'R_ankle', 'R_knee_angle')
        ]

        for p1, p2, p3, feature_name in angle_definitions:
            if all(all(col in df.columns for col in self.landmarks[lm]) for lm in [p1, p2, p3]):

                angle = self._calculate_angle(df, p1, p2, p3)
                if angle is not None:
                    df[feature_name] = angle
                    angles_created.append(feature_name)

        print(f"      ✅ {len(angles_created)} ángulos creados")
        return angles_created

    def _calculate_angle(self, df, point1, point2, point3):
        """Calcular ángulo entre 3 puntos"""
        try:
            # Vectores
            x1, y1 = df[f"{point1}_x"], df[f"{point1}_y"]
            x2, y2 = df[f"{point2}_x"], df[f"{point2}_y"]
            x3, y3 = df[f"{point3}_x"], df[f"{point3}_y"]

            # Vector 1: point2 -> point1
            v1_x, v1_y = x1 - x2, y1 - y2
            # Vector 2: point2 -> point3
            v2_x, v2_y = x3 - x2, y3 - y2

            # Calcular ángulo usando producto punto
            dot_product = v1_x * v2_x + v1_y * v2_y
            magnitude1 = np.sqrt(v1_x**2 + v1_y**2)
            magnitude2 = np.sqrt(v2_x**2 + v2_y**2)

            # Evitar división por cero
            magnitude_product = magnitude1 * magnitude2
            valid_mask = magnitude_product > 1e-8

            angles = np.zeros(len(df))
            angles[valid_mask] = np.arccos(
                np.clip(dot_product[valid_mask] / magnitude_product[valid_mask], -1, 1)
            )

            return np.degrees(angles)  # Convertir a grados

        except Exception as e:
            print(f"❌ Error calculando ángulo: {e}")
            return None

    def _create_ratio_features(self, df):
        """Crear ratios y proporciones corporales"""
        print("   📊 Creando ratios corporales...")

        ratios_created = []

        # Ratios corporales útiles
        if all(col in df.columns for col in ['shoulder_width', 'hip_width']):
            df['shoulder_hip_ratio'] = df['shoulder_width'] / (df['hip_width'] + 1e-8)
            ratios_created.append('shoulder_hip_ratio')

        if all(col in df.columns for col in ['L_torso_length', 'L_thigh_length']):
            df['torso_thigh_ratio'] = df['L_torso_length'] / (df['L_thigh_length'] + 1e-8)
            ratios_created.append('torso_thigh_ratio')

        # Altura aproximada (hombro a tobillo)
        if all(col in df.columns for col in ['L_shoulder_y', 'L_ankle_y']):
            df['body_height_approx'] = abs(df['L_ankle_y'] - df['L_shoulder_y'])
            ratios_created.append('body_height_approx')

        print(f"      ✅ {len(ratios_created)} ratios creados")
        return ratios_created

    def _create_center_mass_features(self, df):
        """Crear características de centro de masa"""
        print("   ⚖️ Creando centro de masa...")

        center_features = []

        # Centro de masa corporal (promedio caderas)
        if all(col in df.columns for col in ['L_hip_x', 'R_hip_x', 'L_hip_y', 'R_hip_y']):
            df['center_mass_x'] = (df['L_hip_x'] + df['R_hip_x']) / 2
            df['center_mass_y'] = (df['L_hip_y'] + df['R_hip_y']) / 2
            center_features.extend(['center_mass_x', 'center_mass_y'])

        # Centro torso superior (promedio hombros)
        if all(col in df.columns for col in ['L_shoulder_x', 'R_shoulder_x', 'L_shoulder_y', 'R_shoulder_y']):
            df['upper_center_x'] = (df['L_shoulder_x'] + df['R_shoulder_x']) / 2
            df['upper_center_y'] = (df['L_shoulder_y'] + df['R_shoulder_y']) / 2
            center_features.extend(['upper_center_x', 'upper_center_y'])

        print(f"      ✅ {len(center_features)} centros de masa creados")
        return center_features

# Crear feature engineer
geo_engineer = GeometricFeatureEngineer()

# Aplicar feature engineering geométrico
if df_augmented is not None:
    df_with_geo_features, geo_features = geo_engineer.create_geometric_features(df_augmented)
    print(f"\n✅ FEATURE ENGINEERING GEOMÉTRICO COMPLETADO")
    print(f"📊 Nuevas features: {len(geo_features)}")
else:
    print("❌ No hay dataset para feature engineering")


🔶 CREANDO CARACTERÍSTICAS GEOMÉTRICAS
   📏 Creando distancias entre landmarks...
      ✅ 8 distancias creadas
   📐 Creando ángulos articulares...
      ✅ 4 ángulos creados
   📊 Creando ratios corporales...
      ✅ 3 ratios creados
   ⚖️ Creando centro de masa...
      ✅ 4 centros de masa creados

✅ CARACTERÍSTICAS GEOMÉTRICAS CREADAS:
   📊 Total nuevas features: 19
   📐 Dimensiones: 11,406 × 88

✅ FEATURE ENGINEERING GEOMÉTRICO COMPLETADO
📊 Nuevas features: 19


## Paso 4: Feature Engineering - Características Temporales  
Crear características temporales (velocidades, aceleraciones) calculadas entre frames consecutivos.


In [22]:
# FEATURE ENGINEERING TEMPORAL - VERSIÓN CORREGIDA (SIN PÉRDIDAS)
class TemporalFeatureEngineerFixed:
    """Crear características temporales SIN perder datos sintéticos"""

    def __init__(self):
        # Landmarks para análisis temporal
        self.key_landmarks = [
            'L_shoulder_y', 'R_shoulder_y',
            'L_hip_y', 'R_hip_y',
            'L_knee_y', 'R_knee_y'
        ]

        # Solo usar landmarks que existan
        self.center_landmarks = ['center_mass_x', 'center_mass_y']

    def create_temporal_features_safe(self, df):
        """Crear features temporales SIN procesar por video individual"""
        print("⏱️ FEATURE ENGINEERING TEMPORAL - MÉTODO SEGURO")
        print("=" * 55)

        df_temporal = df.copy()
        temporal_features_created = []

        print("💡 ESTRATEGIA: Calcular temporales por actividad + source")
        print("🎯 OBJETIVO: Preservar TODOS los datos sintéticos")

        # Landmarks disponibles
        available_landmarks = [lm for lm in self.key_landmarks + self.center_landmarks
                             if lm in df.columns]

        print(f"📊 Landmarks disponibles: {len(available_landmarks)}")

        # Crear características temporales frame-by-frame
        for landmark in available_landmarks:

            # 1. Velocidad aproximada (diferencia con frame anterior)
            df_temporal[f"{landmark}_velocity"] = df[landmark].diff().fillna(0)
            temporal_features_created.append(f"{landmark}_velocity")

            # 2. Aceleración aproximada (diferencia de velocidades)
            velocity_col = f"{landmark}_velocity"
            df_temporal[f"{landmark}_acceleration"] = df_temporal[velocity_col].diff().fillna(0)
            temporal_features_created.append(f"{landmark}_acceleration")

            # 3. Media móvil simple (ventana 3)
            df_temporal[f"{landmark}_smooth"] = df[landmark].rolling(window=3, center=True).mean().fillna(df[landmark])
            temporal_features_created.append(f"{landmark}_smooth")

        # Características estadísticas globales (por actividad)
        print(f"\n📊 Creando características estadísticas...")

        # Variabilidad del centro de masa por grupo
        if 'center_mass_y' in df.columns:
            # Agrupar por actividad para estadísticas
            activity_stats = df.groupby('activity')['center_mass_y'].agg(['mean', 'std']).reset_index()

            # Merge con dataset principal
            df_temporal = df_temporal.merge(
                activity_stats.rename(columns={'mean': 'activity_center_mean', 'std': 'activity_center_std'}),
                on='activity',
                how='left'
            )

            temporal_features_created.extend(['activity_center_mean', 'activity_center_std'])

        print(f"✅ CARACTERÍSTICAS TEMPORALES CREADAS:")
        print(f"   📊 Total features temporales: {len(temporal_features_created)}")
        print(f"   📐 Frames preservados: {len(df_temporal):,} (¡Sin pérdidas!)")

        # Verificar que no se perdieron datos
        if len(df_temporal) != len(df):
            print(f"⚠️ ADVERTENCIA: Se perdieron {len(df) - len(df_temporal):,} frames")
        else:
            print(f"✅ PERFECTO: Todos los frames preservados")

        return df_temporal, temporal_features_created

# Aplicar temporal feature engineering CORREGIDO
temporal_engineer_fixed = TemporalFeatureEngineerFixed()

if 'df_with_geo_features' in locals() and df_with_geo_features is not None:
    df_temporal_fixed, temporal_features_fixed = temporal_engineer_fixed.create_temporal_features_safe(df_with_geo_features)

    print(f"\n✅ TEMPORAL FEATURE ENGINEERING CORREGIDO")
    print(f"📊 Features: {len(temporal_features_fixed)}")
    print(f"🎯 Samples preservados: {len(df_temporal_fixed):,}")
else:
    print("❌ Dataset con features geométricas no disponible")


⏱️ FEATURE ENGINEERING TEMPORAL - MÉTODO SEGURO
💡 ESTRATEGIA: Calcular temporales por actividad + source
🎯 OBJETIVO: Preservar TODOS los datos sintéticos
📊 Landmarks disponibles: 8

📊 Creando características estadísticas...
✅ CARACTERÍSTICAS TEMPORALES CREADAS:
   📊 Total features temporales: 26
   📐 Frames preservados: 11,406 (¡Sin pérdidas!)
✅ PERFECTO: Todos los frames preservados

✅ TEMPORAL FEATURE ENGINEERING CORREGIDO
📊 Features: 26
🎯 Samples preservados: 11,406


## Paso 5: Pipeline de Preprocessing
Crear pipeline automatizado para normalización, encoding y preparación final.


In [23]:
# PIPELINE DE PREPROCESSING - VERSIÓN CORREGIDA
class DataPreprocessingPipelineFixed:
    """Pipeline completo preservando todos los datos aumentados"""

    def __init__(self):
        self.scaler = StandardScaler()
        self.label_encoder = LabelEncoder()
        self.pca = None
        self.preprocessing_stats = {}

    def create_ml_ready_dataset(self, df, use_pca=True, n_components=0.95):
        """Crear dataset final con TODOS los datos preservados"""
        print("🔧 PIPELINE DE PREPROCESSING - SIN PÉRDIDAS")
        print("=" * 50)

        print(f"📊 DATASET DE ENTRADA:")
        print(f"   Total frames: {len(df):,}")
        print(f"   Columnas: {len(df.columns)}")

        # Verificar distribución por fuente
        if 'source' in df.columns:
            print(f"   Por fuente:")
            for source in df['source'].unique():
                count = len(df[df['source'] == source])
                print(f"      {source}: {count:,} frames")

        # 1. Separar features y targets CUIDADOSAMENTE
        X, y, metadata = self._separate_features_targets_safe(df)

        if X is None:
            print("❌ Error separando features")
            return None

        print(f"\n📊 SEPARACIÓN DE DATOS:")
        print(f"   ✅ Features (X): {X.shape} - PRESERVADAS")
        print(f"   ✅ Labels (y): {len(y):,} - PRESERVADAS")
        print(f"   ✅ Metadata: {len(metadata) if metadata is not None else 0:,} - PRESERVADA")

        # 2. Encoding de labels
        y_encoded = self.label_encoder.fit_transform(y)

        print(f"\n🏷️ LABEL ENCODING:")
        for i, class_name in enumerate(self.label_encoder.classes_):
            count = np.sum(y_encoded == i)
            print(f"   {class_name.replace('_', ' ').title()}: {count:,} samples (código {i})")

        # 3. Limpiar features (eliminar NaN, infinitos)
        print(f"\n🧹 LIMPIEZA DE FEATURES:")

        # Verificar NaN
        nan_counts = np.isnan(X).sum(axis=0)
        cols_with_nan = np.where(nan_counts > 0)[0]

        if len(cols_with_nan) > 0:
            print(f"   ⚠️ Columnas con NaN: {len(cols_with_nan)}")
            X = np.nan_to_num(X, nan=0.0, posinf=1.0, neginf=-1.0)
            print(f"   ✅ NaN/Inf reemplazados con valores seguros")
        else:
            print(f"   ✅ Sin valores NaN - datos limpios")

        # 4. Normalización con StandardScaler
        X_scaled = self.scaler.fit_transform(X)
        print(f"   ✅ StandardScaler aplicado")

        # 5. PCA para reducción dimensional
        if use_pca:
            self.pca = PCA(n_components=n_components, random_state=42)
            X_pca = self.pca.fit_transform(X_scaled)

            n_components_final = X_pca.shape[1]
            variance_explained = self.pca.explained_variance_ratio_.sum()

            print(f"   ✅ PCA: {X.shape[1]} → {n_components_final} features")
            print(f"   📊 Varianza: {variance_explained*100:.1f}%")

            X_final = X_pca
        else:
            X_final = X_scaled
            n_components_final = X_scaled.shape[1]
            variance_explained = 1.0

        # 6. Estadísticas finales del pipeline
        self.preprocessing_stats = {
            'input_samples': len(df),
            'output_samples': X_final.shape[0],
            'preservation_rate': X_final.shape[0] / len(df),
            'original_features': X.shape[1],
            'final_features': X_final.shape[1],
            'reduction_rate': (X.shape[1] - X_final.shape[1]) / X.shape[1],
            'classes': len(self.label_encoder.classes_),
            'variance_explained': variance_explained
        }

        print(f"\n📊 PIPELINE COMPLETADO:")
        print(f"   📈 Preservación: {self.preprocessing_stats['preservation_rate']*100:.1f}%")
        print(f"   📉 Reducción features: {self.preprocessing_stats['reduction_rate']*100:.1f}%")
        print(f"   🎯 Lista para ML: {X_final.shape}")

        return X_final, y_encoded, metadata

    def _separate_features_targets_safe(self, df):
        """Separar features y targets preservando todos los datos"""
        # Columnas no-feature (metadata)
        metadata_cols = ['activity', 'video_file', 'frame_number', 'source', 'augmentation_type']

        # Features = todas las demás columnas
        feature_cols = [col for col in df.columns if col not in metadata_cols]

        print(f"   🔍 Identificadas {len(feature_cols)} columnas de features")
        print(f"   📋 Metadata: {len([col for col in metadata_cols if col in df.columns])} columnas")

        # Verificar que tenemos datos válidos
        if not feature_cols:
            print("❌ No hay columnas de features")
            return None, None, None

        if 'activity' not in df.columns:
            print("❌ No hay columna 'activity'")
            return None, None, None

        # Extraer datos
        X = df[feature_cols].values  # Convertir a numpy
        y = df['activity'].values

        # Metadata disponible
        available_metadata = [col for col in metadata_cols if col in df.columns]
        metadata = df[available_metadata] if available_metadata else None

        return X, y, metadata

# Crear pipeline corregido
preprocessing_pipeline_fixed = DataPreprocessingPipelineFixed()

# Aplicar preprocessing SIN pérdidas
if 'df_temporal_fixed' in locals() and df_temporal_fixed is not None:
    X_processed_fixed, y_processed_fixed, metadata_fixed = preprocessing_pipeline_fixed.create_ml_ready_dataset(
        df_temporal_fixed,
        use_pca=True,
        n_components=0.95
    )

    if X_processed_fixed is not None:
        print(f"\n🎉 PREPROCESSING CORREGIDO COMPLETADO")
        print(f"✅ Dataset ML-ready: {X_processed_fixed.shape}")
        print(f"✅ TODOS los 11,406 frames preservados en pipeline")
    else:
        print("❌ Error en preprocessing corregido")
else:
    print("❌ Dataset temporal corregido no disponible")


🔧 PIPELINE DE PREPROCESSING - SIN PÉRDIDAS
📊 DATASET DE ENTRADA:
   Total frames: 11,406
   Columnas: 114
   Por fuente:
      real_video: 4,575 frames
      smote_synthetic: 5,445 frames
      spatial_augmentation: 500 frames
      temporal_augmentation: 886 frames
   🔍 Identificadas 109 columnas de features
   📋 Metadata: 5 columnas

📊 SEPARACIÓN DE DATOS:
   ✅ Features (X): (11406, 109) - PRESERVADAS
   ✅ Labels (y): 11,406 - PRESERVADAS
   ✅ Metadata: 11,406 - PRESERVADA

🏷️ LABEL ENCODING:
   Caminar Hacia: 2,464 samples (código 0)
   Caminar Regreso: 2,368 samples (código 1)
   Girar: 2,278 samples (código 2)
   Ponerse Pie: 2,227 samples (código 3)
   Sentarse: 2,069 samples (código 4)

🧹 LIMPIEZA DE FEATURES:
   ✅ Sin valores NaN - datos limpios
   ✅ StandardScaler aplicado
   ✅ PCA: 109 → 19 features
   📊 Varianza: 95.1%

📊 PIPELINE COMPLETADO:
   📈 Preservación: 100.0%
   📉 Reducción features: 82.6%
   🎯 Lista para ML: (11406, 19)

🎉 PREPROCESSING CORREGIDO COMPLETADO
✅ Datas

## Paso 6: Train/Validation/Test Splits Estratificados
Crear splits finales preservando balance de clases en cada partición.


In [28]:
# CREAR SPLITS ESTRATIFICADOS FINALES
def create_final_stratified_splits(X, y, metadata=None):
    """Crear splits estratificados preservando balance"""
    print("📋 CREANDO SPLITS ESTRATIFICADOS FINALES")
    print("=" * 50)

    print(f"📊 DATASET COMPLETO PARA SPLITS:")
    print(f"   📈 Total samples: {X.shape[0]:,}")
    print(f"   🔢 Features: {X.shape[1]}")
    print(f"   🎯 Classes: {len(np.unique(y))}")

    # Crear splits: 70% / 15% / 15%
    X_trainval, X_test, y_trainval, y_test = train_test_split(
        X, y,
        test_size=0.15,
        stratify=y,
        random_state=42
    )

    X_train, X_val, y_train, y_val = train_test_split(
        X_trainval, y_trainval,
        test_size=0.176,  # 15/85 ≈ 0.176
        stratify=y_trainval,
        random_state=42
    )

    print(f"   ✅ Train: {X_train.shape[0]:,} samples ({X_train.shape[0]/X.shape[0]*100:.1f}%)")
    print(f"   ✅ Validation: {X_val.shape[0]:,} samples ({X_val.shape[0]/X.shape[0]*100:.1f}%)")
    print(f"   ✅ Test: {X_test.shape[0]:,} samples ({X_test.shape[0]/X.shape[0]*100:.1f}%)")

    # Verificar balance en cada split
    splits_to_check = [('Train', y_train), ('Validation', y_val), ('Test', y_test)]

    for split_name, y_split in splits_to_check:
        unique_split, counts_split = np.unique(y_split, return_counts=True)
        balance_split = counts_split.min() / counts_split.max()
        print(f"   {split_name}: Balance {balance_split:.3f}")

    return {
        'train': (X_train, y_train),
        'validation': (X_val, y_val),
        'test': (X_test, y_test)
    }

# EJECUTAR SPLITS (usando las variables correctas)
final_data_splits = create_final_stratified_splits(
    X_processed_fixed,
    y_processed_fixed
)

print(f"✅ SPLITS CREADOS EXITOSAMENTE")


📋 CREANDO SPLITS ESTRATIFICADOS FINALES
📊 DATASET COMPLETO PARA SPLITS:
   📈 Total samples: 11,406
   🔢 Features: 19
   🎯 Classes: 5
   ✅ Train: 7,988 samples (70.0%)
   ✅ Validation: 1,707 samples (15.0%)
   ✅ Test: 1,711 samples (15.0%)
   Train: Balance 0.840
   Validation: Balance 0.840
   Test: Balance 0.838
✅ SPLITS CREADOS EXITOSAMENTE


In [36]:
# GUARDADO SIMPLIFICADO - DIRECTO
def save_datasets_direct():
    """Guardar datasets directamente"""
    print("💾 GUARDANDO DATASETS - MÉTODO DIRECTO")
    print("=" * 45)

    # Crear directorios
    os.makedirs('data/processed', exist_ok=True)
    os.makedirs('data/models', exist_ok=True)

    # Guardar splits
    for split_name, (X_split, y_split) in final_data_splits.items():
        np.save(f'data/processed/X_{split_name}.npy', X_split)
        np.save(f'data/processed/y_{split_name}.npy', y_split)
        print(f"   ✅ {split_name}: {X_split.shape[0]:,} samples guardados")

    # Guardar transformers
    joblib.dump(preprocessing_pipeline_fixed.scaler, 'data/models/scaler.pkl')
    joblib.dump(preprocessing_pipeline_fixed.label_encoder, 'data/models/encoder.pkl')
    joblib.dump(preprocessing_pipeline_fixed.pca, 'data/models/pca.pkl')

    print(f"   ✅ Transformers guardados")

    # Guardar dataset completo procesado
    np.save('data/processed/X_complete.npy', X_processed_fixed)
    np.save('data/processed/y_complete.npy', y_processed_fixed)

    print(f"\n🎉 GUARDADO COMPLETADO:")
    print(f"   📊 {len(final_data_splits)} splits creados")
    print(f"   🔧 3 transformers guardados")
    print(f"   📁 Dataset completo disponible")

    return True

# Guardar todo
save_success = save_datasets_direct()


💾 GUARDANDO DATASETS - MÉTODO DIRECTO
   ✅ train: 7,988 samples guardados
   ✅ validation: 1,707 samples guardados
   ✅ test: 1,711 samples guardados
   ✅ Transformers guardados

🎉 GUARDADO COMPLETADO:
   📊 3 splits creados
   🔧 3 transformers guardados
   📁 Dataset completo disponible
